In [120]:
!pip install absl-py

You should consider upgrading via the 'C:\Users\nurha\.conda\envs\audio_env\python.exe -m pip install --upgrade pip' command.


In [121]:
import os
import time
import shutil
import librosa
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm_notebook
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score, roc_curve\
,auc
import matplotlib.pyplot as plt


import tensorflow as tf
print(f'tensorflow version: {tf.__version__}')
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout, Conv2D, MaxPooling2D, BatchNormalization, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping

tensorflow version: 2.7.0


# Import Data

In [148]:
df = pd.read_pickle('28k_cleaned.pkl')

In [149]:
df['um_label'].value_counts()

0.0    25761
1.0     6092
Name: um_label, dtype: int64

In [150]:
X = np.stack(df['mfcc'].to_numpy())
y = df['um_label'].to_numpy().reshape(-1,1).astype('float32')
print(X.shape)
print(y.shape)

(31853, 13, 94)
(31853, 1)


In [151]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=14)

input_shape = (X_train.shape[1], X_train.shape[2], 1)
input_shape

(13, 94, 1)

In [165]:
def create_cnn_model(input_shape):
    model = Sequential()

    model.add(Conv2D(32, (3,3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((3,3), strides=(2, 2), padding='same'))
    model.add(BatchNormalization())

    model.add(Conv2D(64, (3,3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((3,3), strides=(2, 2), padding='same'))
    model.add(BatchNormalization())

    model.add(Conv2D(128, (2,2), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2,2), strides=(2, 2), padding='same'))
    model.add(BatchNormalization())

    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(1, activation='sigmoid'))
    
    return model

In [159]:
model = create_cnn_model(input_shape)

In [160]:
adam_optimizer = Adam(learning_rate=0.0001)

model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=adam_optimizer)

In [162]:
%%time
num_epochs = 100
num_batch_size = 32

model_checkpoint = ModelCheckpoint(filepath='model4.hdf5', 
                               verbose=1, save_best_only=True)

early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=15)

history = model.fit(X_train, 
          y_train, 
          batch_size=num_batch_size, 
          epochs=num_epochs, 
          validation_data=(X_test, y_test), 
          callbacks=[model_checkpoint, early_stop], 
          verbose=1)

Epoch 1/100
796/797 [============================>.] - ETA: 0s - loss: 0.5668 - accuracy: 0.7515
Epoch 00001: val_loss improved from inf to 0.43593, saving model to model4.hdf5
797/797 [==============================] - 17s 21ms/step - loss: 0.5668 - accuracy: 0.7516 - val_loss: 0.4359 - val_accuracy: 0.8132
Epoch 2/100
794/797 [============================>.] - ETA: 0s - loss: 0.4825 - accuracy: 0.7995
Epoch 00002: val_loss improved from 0.43593 to 0.41219, saving model to model4.hdf5
797/797 [==============================] - 17s 21ms/step - loss: 0.4822 - accuracy: 0.7996 - val_loss: 0.4122 - val_accuracy: 0.8239
Epoch 3/100
794/797 [============================>.] - ETA: 0s - loss: 0.4423 - accuracy: 0.8186
Epoch 00003: val_loss improved from 0.41219 to 0.37693, saving model to model4.hdf5
797/797 [==============================] - 17s 21ms/step - loss: 0.4428 - accuracy: 0.8183 - val_loss: 0.3769 - val_accuracy: 0.8401
Epoch 4/100
796/797 [============================>.] - ETA: 0s

Epoch 29/100
795/797 [============================>.] - ETA: 0s - loss: 0.1283 - accuracy: 0.9477
Epoch 00029: val_loss did not improve from 0.26357
797/797 [==============================] - 16s 21ms/step - loss: 0.1283 - accuracy: 0.9478 - val_loss: 0.3456 - val_accuracy: 0.8994
Epoch 30/100
794/797 [============================>.] - ETA: 0s - loss: 0.1233 - accuracy: 0.9516
Epoch 00030: val_loss did not improve from 0.26357
797/797 [==============================] - 16s 21ms/step - loss: 0.1233 - accuracy: 0.9515 - val_loss: 0.3781 - val_accuracy: 0.8953
Epoch 00030: early stopping
CPU times: total: 58min 7s
Wall time: 8min 17s


# Test Model

In [168]:
model = create_cnn_model(input_shape)

model.load_weights('models/v4.hdf5')

In [169]:
def test_thresholds(X, y):
    fpr, tpr, thresholds = roc_curve(y, model.predict(X))
    plt.figure()
    lw = 2
    plt.plot(
        fpr,
        tpr,
        color="darkorange",
        lw=lw,
        label="ROC curve (area = %0.2f)" % auc(fpr, tpr),
    )
    plt.plot([0, 1], [0, 1], color="navy", lw=lw, linestyle="--")
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("Receiver operating characteristic example")
    plt.legend(loc="lower right")
    plt.show()
    
    possible_thresholds = np.linspace(0, 1, num=50)
    df_metrics = pd.DataFrame()
    for threshold in possible_thresholds:
        y_pred = (model.predict(X) > threshold).astype("int32")
        df_metrics = df_metrics.append({
            'threshold': threshold,
            'accuracy': accuracy_score(y, y_pred),
            'f1_score': f1_score(y, y_pred)
        }, ignore_index=True)
        
    sns.lineplot(data=df_metrics.set_index('threshold'), palette="tab10", linewidth=2.5)
    
    return df_metrics

In [173]:
y_pred = (model.predict(X_test) > 0.5).astype("int32")
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(f'f1 score: {f1_score(y_test, y_pred)}')
confusion_matrix(y_test, y_pred)

Accuracy: 0.9037827656568828
f1 score: 0.7136851938346568


array([[4994,  177],
       [ 436,  764]], dtype=int64)